In [1]:
from collections import namedtuple
import geopandas as gpd
import matplotlib.pyplot as plot
from shapely import geometry
import pandas as pd
import itertools
from collections import deque
import numpy as np

In [5]:
routes = gpd.read_file('./data/poor_10000.shp')
routes=routes.to_crs('EPSG:4326')
district=gpd.read_file('./data/poor_zone.shp')

In [6]:
def getFnidByPoint(first_point,second_point):
    """
    return the fnid of each line

    Args:
        first_point : start position of line
        second_point : end position of line

    Returns:
        fnid_list: fnid list
    """    
    fnid_list=[]
    geometry_first_point=gpd.GeoSeries(geometry.Point(first_point),crs=4326)
    geometry_second_point=gpd.GeoSeries(geometry.Point(second_point),crs=4326)
    first_point_=gpd.GeoDataFrame(geometry=geometry_first_point,crs=4326)
    second_point_=gpd.GeoDataFrame(geometry=geometry_second_point,crs=4326)
    intersect_first=gpd.overlay(df1=district,df2=first_point_,how='intersection',keep_geom_type=False)
    fnid_list.extend(list(intersect_first['fnid']))
    # if distance>250m,then interpolation,
    # the projection coordinate system of ShenZhen is UTM 50N(EPSG:3406)
    UTM_first_point = geometry_first_point.to_crs(32650)
    UTM_second_point=geometry_second_point.to_crs(32650)
    dist=UTM_first_point.distance(UTM_second_point)
    delta=int(dist//50)
    for i in range(delta):
        interpolation_x=first_point[0]+i*(second_point[0]-first_point[0])/delta
        interpolation_y=first_point[1]+i*(second_point[1]-first_point[1])/delta
        geometry_interpolation=gpd.GeoSeries(geometry.Point([interpolation_x,interpolation_y]),crs=4326)
        interpolation_point=gpd.GeoDataFrame(geometry=geometry_interpolation,crs=4326)
        intersect_=gpd.overlay(df1=district,df2=interpolation_point,how='intersection',keep_geom_type=False)
        fnid_list.extend(list(intersect_['fnid']))
    intersect_second=gpd.overlay(df1=district,df2=second_point_,how='intersection',keep_geom_type=False)
    fnid_list.extend(list(intersect_second['fnid']))
    return fnid_list

In [7]:
length=len(routes)
print('the length of routes is {}'.format(length))
routes_states=[]
for i in range(length):
    geometry_=routes.iloc[i,-1]
    q=deque(geometry_.coords)
    len_q=len(q)
    first_point=q.popleft()
    states=[]
    states_unique=[]
    while q:
        second_point=q.popleft()
        fnid_list=getFnidByPoint(first_point,second_point)
        if fnid_list:
            states.extend(fnid_list)
        first_point=second_point
    for key,_ in itertools.groupby(states):
        states_unique.append(key)
    print("the {} route finished".format(i))
    routes_states.append(states_unique)

the length of routes is 617
the 0 route finished
the 1 route finished
the 2 route finished
the 3 route finished
the 4 route finished
the 5 route finished
the 6 route finished
the 7 route finished
the 8 route finished
the 9 route finished
the 10 route finished
the 11 route finished
the 12 route finished
the 13 route finished
the 14 route finished
the 15 route finished
the 16 route finished
the 17 route finished
the 18 route finished
the 19 route finished
the 20 route finished
the 21 route finished
the 22 route finished
the 23 route finished
the 24 route finished
the 25 route finished
the 26 route finished
the 27 route finished
the 28 route finished
the 29 route finished
the 30 route finished
the 31 route finished
the 32 route finished
the 33 route finished
the 34 route finished
the 35 route finished
the 36 route finished
the 37 route finished
the 38 route finished
the 39 route finished
the 40 route finished
the 41 route finished
the 42 route finished
the 43 route finished
the 44 route f

In [8]:
routes_states=np.array(routes_states)
np.save('./data/routes_states_poor.npy',routes_states) 
routes_states=np.load('./data/routes_states_poor.npy',allow_pickle=True)
print(np.load('./data/routes_states_poor.npy',allow_pickle=True)==routes_states)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

e:\Anaconda\envs\django\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [9]:
actions = [0, 1, 2, 3, 4]
dirs = {0: 'r', 1: 'l', 2: 'd', 3: 'u', 4: 's'}
#        right,    left,   down,   up ,   stay
Step=namedtuple('Step',['state','action'])
def getActionOfStates(route_state):
    state_action=[]
    length=len(route_state)
    first_state=route_state[0]
    if length==1:
        step=Step(state=first_state,action=4)
        state_action.append(step)
        return state_action
    for i in range(1,length):
        second_state=route_state[i]

        def getAction(first_s,second_s):
            if second_s-first_s==1:
                return 0
            elif second_s- first_s==-1:
                return 1
            elif second_s-first_s==357:
                return 3
            elif second_s-first_s==-357:
                return 2

        idx_minux=second_state-first_state
        if idx_minux==358:
            second_state=second_state-1
            act=getAction(first_state,second_state)
            step=Step(state=first_state,action=act)
            state_action.append(step)
            first_state=second_state
            second_state+=1
        elif idx_minux==356:
            second_state=second_state+1
            act=getAction(first_state,second_state)
            step=Step(state=first_state,action=act)
            state_action.append(step)
            first_state=second_state
            second_state-=1
        elif idx_minux==-358:
            second_state=second_state+1
            act=getAction(first_state,second_state)
            step=Step(state=first_state,action=act)
            state_action.append(step)
            first_state=second_state
            second_state-=1
        elif idx_minux==-356:
            second_state=second_state-1
            act=getAction(first_state,second_state)
            step=Step(state=first_state,action=act)
            state_action.append(step)
            first_state=second_state
            second_state+=1
        act=getAction(first_state,second_state)
        step=Step(state=first_state,action=act)
        state_action.append(step)
        first_state=second_state
    step=Step(state=second_state,action=4)
    state_action.append(step)
    return state_action

In [10]:
state_action_tuple=[]
for route_state in routes_states:
    sta_act=getActionOfStates(route_state)
    state_action_tuple.append(sta_act)
state_action_tuple[0]

[Step(state=39417.0, action=3),
 Step(state=39774.0, action=0),
 Step(state=39775.0, action=2),
 Step(state=39418.0, action=0),
 Step(state=39419.0, action=0),
 Step(state=39420.0, action=3),
 Step(state=39777.0, action=3),
 Step(state=40134.0, action=0),
 Step(state=40135.0, action=3),
 Step(state=40492.0, action=0),
 Step(state=40493.0, action=0),
 Step(state=40494.0, action=2),
 Step(state=40137.0, action=0),
 Step(state=40138.0, action=3),
 Step(state=40495.0, action=4)]

In [11]:
state_action_tuple=np.array(state_action_tuple)
np.save('./data/state_action_tuple_poor.npy',state_action_tuple) 
print(np.load('./data/state_action_tuple_poor.npy',allow_pickle=True)==state_action_tuple)

e:\Anaconda\envs\django\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T